In [1]:
#!pip3 install praw

     |████████████████████████████████| 188 kB 39.2 MB/s 


     |██████████████████████████████▏ | 51 kB 61.1 MB/s eta 0:00:01

     |████████████████████████████████| 54 kB 2.8 MB/s 


Go to the following link here _:_

https://www.reddit.com/prefs/apps/



In [19]:
import praw,requests,math, itertools,collections, wordcloud
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /projects/0ddeade5-3577-4
[nltk_data]     fe8-8cd6-8a0cb653428e/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
reddit = praw.Reddit(
    client_id='lgx1EAJJQJlAu_7rCAoBrg', 
    client_secret='OkQIm3ny_AFQ3LzIyhi8gEwXgY6Ydw',
    user_agent='dadjokes',
    username='wiytre',
    password='ooferonI!1')

In [6]:
print(reddit.user.me())


wiytre


#### Finding results from a subreddit

We use the query reddit.subreddit\('&lt;\-\- Subreddit Name \-\-&gt;'\).hot\(limit = 5\).

hot indicates, in terms of reddit, the trending posts

Other alternatives for hot are new \( latest posts \) and rising \( posts which getting more activity \)\(limit = 5\) indicates we are selecting 5 postsIn


In [10]:
rising_posts = reddit.subreddit('dadjokes').rising(limit=1)
for post in rising_posts:
    print(post.title,end="\n\n")

Hold your loved ones close



In [11]:
hot_posts = reddit.subreddit('dadjokes').hot(limit=1)
for post in hot_posts:
    print(post.title,end="\n\n")

My wife left me a note on the fridge saying “it’s not working, I’m at my parents place”



#### Choosing what information we want

title \-&gt; We get the title of reddit post

selftext \-&gt; We get the content of reddit post

author \-&gt; We get the author of reddit post

ups \-&gt; We get the upvotes for the reddit post

downs \-&gt; We get the downvotes for the reddit post

num\_comments \-&gt; We get the number of comments on reddit post

link\_flair\_text \-&gt; We get the tags if any on the post

url \-&gt; We get the url if any of the reddit post

We will store that data in pandas dataframe


In [30]:
def remove_stopwords(sentences):
    for i in range(len(sentences)):
        words = nltk.word_tokenize(sentences[i])
        words = [word for word in words if word not in stopwords.words('english')]
        sentences[i] = ' '.join(words)
    return sentences

def remove_punctuation(sentences):
    for i in range(len(sentences)):
        words = nltk.word_tokenize(sentences[i])
        words = [word for word in words if word not in "',.[]?!()%{}"]
        sentences[i] = ' '.join(words)
    return sentences

def lem_sentences(sentences):
    for w in range(len(sentences)):
        words = nltk.word_tokenize(sentences[w])
        words = [lemmatizer.lemmatize(w) for w in words]
        sentences[w] = ' '.join(words)
    return sentences

In [34]:
text = pd.read_csv("dadjokes.csv")

In [12]:
hot_posts = reddit.subreddit('dadjokes').hot(limit=2000)

posts = []

for post in hot_posts:
    posts.append([post.title, post.score, post.id, post.selftext, post.author, post.url, post.num_comments, 
                  post.ups, post.downs, post.link_flair_text])

posts_df = pd.DataFrame(posts,columns=['title', 'score', 'id', 'selftext', 'author', 'url', 'num_comments',
                                       'upvotes','downvotes','category'])

#Separating the news provider from the link
posts_df['dad_jokes'] = posts_df['url'].str.split('/').str[2]

In [13]:
posts_df.head()

,title,score,id,selftext,author,url,num_comments,upvotes,downvotes,category,dad_jokes
0,My wife left me a note on the fridge saying “i...,2633,vb97xv,"After opening the fridge, I’ve realized it was...",kdesign,https://www.reddit.com/r/dadjokes/comments/vb9...,90,2633,0,None,www.reddit.com
1,My girlfriend is leaving me saying I am not Am...,10880,vb01hy,Saw it coming a kilometer away.,clitsdontexist,https://www.reddit.com/r/dadjokes/comments/vb0...,229,10880,0,None,www.reddit.com
2,an unexpected emotional journey,479,vbcl0j,My cousin (MtF) has just come out to the famil...,dopeaminenotanime,https://www.reddit.com/r/dadjokes/comments/vbc...,20,479,0,None,www.reddit.com
3,My wife claims that men in camouflage look rea...,604,vbafz5,I just….don’t see it.,porichoygupto,https://www.reddit.com/r/dadjokes/comments/vba...,11,604,0,None,www.reddit.com
4,How do you please a woman in bed?,151,vbbvis,Let her sleep.,VERBERD,https://www.reddit.com/r/dadjokes/comments/vbb...,13,151,0,None,www.reddit.com


In [14]:
posts_df.sort_values(by='score', ascending=False).head(5)

,title,score,id,selftext,author,url,num_comments,upvotes,downvotes,category,dad_jokes
1,My girlfriend is leaving me saying I am not Am...,10880,vb01hy,Saw it coming a kilometer away.,clitsdontexist,https://www.reddit.com/r/dadjokes/comments/vb0...,229,10880,0,None,www.reddit.com
357,This sub is really going downhill,8358,v8to3w,https://i.imgur.com/LTic1Aq.jpg,chad-beer-316,https://www.reddit.com/r/dadjokes/comments/v8t...,144,8358,0,None,www.reddit.com
492,A horse walks into a bar and orders a pint of ...,8186,v85t7c,"The bartender says, “You come in here pretty o...",docmoonlight,https://www.reddit.com/r/dadjokes/comments/v85...,255,8186,0,None,www.reddit.com
743,I dumped my last girlfriend because she was a ...,7285,v6u0oj,I should've known sooner. There were red flags...,scott3845,https://www.reddit.com/r/dadjokes/comments/v6u...,185,7285,0,None,www.reddit.com
824,"What’s the difference between a tuna fish, a p...",7163,v6jj7e,"You can’t tuna fish, but you can tuna piano…",DarlingLee,https://www.reddit.com/r/dadjokes/comments/v6j...,312,7163,0,None,www.reddit.com


In [16]:
posts_df['dadjokes'] = posts_df['title'] + posts_df['selftext']
posts_df['dadjokes'].to_csv('dadjokes.csv', index=False)

In [31]:
remove_stopwords(dadjokes)
remove_punctuation(dadjokes)

KeyError: 0